# Trainver v5 code validation

Simple minimal training runs, to validate v5 code

> Important note: These example focuses only on how to configure your dataset, and does not properly perform checkmarking - for trainer configurations refer to the training notebooks

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="trainer-v5-validation L6-D512"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

## Intial setup

Before we go into the dataset setup, lets perform an initial setup for all the folders we need, and a small toy model which we would use throughout the various examples within this notebook.

In [ ]:
# Setup the folders we will need
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/

# Initialized a simple L6-D512 model
!cd ../../RWKV-v5/ && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size neox --skip-if-exists ../model/L6-D512-v5-neox-init.pth


## Quick train for v5

Preload and train the mini-v5 model

In [ ]:
# Lets preload the requried dataset
!cd ../../RWKV-v5 && \
    python3 preload_datapath.py ../notebook/trainer-x-validation/mini-v5-enwiki.yaml

In [ ]:
# Validate the dataset is working, by doing a quick training run
!cd ../../RWKV-v5 && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-x-validation/mini-v5-enwiki.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (full, train-ctx=4096, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

In [ ]:
# Lets convert the model
!cd ../../RWKV-v5 && \
    python3 export_checkpoint.py \
        ../checkpoint/trainer-validation/mini-v5-enwiki/last.ckpt/ \
        ../model/mini-v5-enwiki.pth

In [ ]:
# And run the dragon test
!cd ../../RWKV-v5 && \
    python3 dragon_test.py ../model/mini-v5-enwiki.pth